In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [2]:
import datetime
from utils import get_path, DATETIME_FORMAT

In [3]:
sql = """
select a.id appoint_id,
	   a.patient_id,
	   concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
	   p_patient.phone,
	   p_patient.email,
	   date(a.finished at time zone 'UTC') consultation_date, 
	   a.finished - a.started consultation_duration,
	   --
	   concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
		case when doctor.profession_name is not null 
			then doctor.profession_name
			else 'Дежурный врач'
			end doctor_profession,
	   --
	   review.rating,
	   review."comment",
	   --
	   case when pc.product_id in (
            select id from product
            where full_name like '%ВЭБ%') then 'VEBMED'
        	else 'ПРОЧИЕ'
       end "name"
from appointment a
left join patient p on a.patient_id = p.id
left join person p_patient on p.person_id = p_patient.id
left join promotion promo on a.promotion_id = promo.id
----
left join product_condition pc
   on promo.product_condition_id = pc.id
----
left join doctor on a.doctor_id = doctor.id 
left join person p_doctor on doctor.person_id = p_doctor.id 
--
left join review on review.appointment_id = a.id
--  
where a.finished at time zone 'UTC' > '2020-03-30'
and a.good
	and not (
			a.patient_id in (
							select patient_id from patient_categories pcat
							inner join reference rf
							on rf.id = pcat.reference_id
							where rf.code = 'TEST'
			)
			or lower(a.report_comment) = 'тест'
	)
order by consultation_date desc	
	
	
	
	
"""

In [4]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-05-26 14:59:56 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-26 14:59:56 creating ssh tunnel to 172.16.100.19 as root...
2020-05-26 14:59:56 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 33495
2020-05-26 14:59:56 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-26 14:59:56 @telemed query:
                    select a.id appoint_id,
				    	   a.patient_id,
				    	   concat_ws(' ', p_patient.last_name, p_patient.first_name, p_patient.middle_name) patient_fio,
				    	   p_patient.phone,
				    	   p_patient.email,
				    	   date(a.finished at time zone 'UTC') consultation_date, 
				    	   a.finished - a.started consultation_duration,
				    	   --
				    	   concat_ws(' ', p_doctor.last_name, p_doctor.first_name, p_doctor.middle_name) doctor_fio,
				    		case when doctor

In [5]:
df = pd.DataFrame(data).copy()

In [6]:
df.head(15)

,appoint_id,patient_id,patient_fio,phone,email,consultation_date,consultation_duration,doctor_fio,doctor_profession,rating,comment,name
0,229387,3749714,Родмина Елена Владимировна,79247056859,None,2020-05-26,00:00:09.741000,Катаева Елена Геннадьевна,Невролог,4.0,None,VEBMED
1,230563,1587086,Коршунова Юлия Артуровна,79005704707,None,2020-05-26,00:18:45.856000,Цыганов Александр Иванович,Дежурный врач,NaN,None,ПРОЧИЕ
2,230239,3488170,Ивлева Вера Юрьевна,79109894929,vera37.57@mail.ru,2020-05-26,00:33:48.522000,Тыченко Ирина Викторовна,Дежурный врач,5.0,Спасибо за внимание и квалифицированную помощь.,VEBMED
3,225975,3750363,Шушкина Людмила Владимировна,79854129100,None,2020-05-26,00:17:31.288000,Новиков Олег Анатольевич,УЗИ,NaN,None,ПРОЧИЕ
4,216851,3518851,Лапина оксана Васильевна,79106890147,None,2020-05-26,00:06:29.808000,Коршенко Элина Валерьевна,Дежурный врач,5.0,None,VEBMED
5,226222,3583666,Рамазов Алексей Николаевич,79634261990,,2020-05-26,00:02:52.132000,Гаджиев Нариман Герейханович,Дежурный врач,NaN,None,VEBMED
6,227022,3587962,Сытова Анна Павловна,79023943962,sva.dba-kaluga@yandex.ru,2020-05-26,00:40:46.843000,Ташинова Елена Сергеевна,Дежурный врач,5.0,None,VEBMED
7,229087,3587676,Ткачук Анастасия Олеговна,79956918349,None,2020-05-26,12:22:35.507000,Субботина Елена Юрьевна,Дежурный врач,NaN,None,VEBMED
8,229377,3754112,Титов Дмитрий Владимирович,79857661184,dmitry.titov@keysight.com,2020-05-26,00:22:19.159000,Грасс Денис Александрович,Дежурный врач,5.0,None,VEBMED
9,229815,3747348,Черясова Светлана Викторовна,79173077759,swetlana.cheryasova@yandex.ry,2020-05-26,00:51:01.741000,Морозова Елизавета Сергеевна,Оториноларинголог,NaN,None,VEBMED


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85335 entries, 0 to 85334
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   appoint_id             85335 non-null  int64          
 1   patient_id             85335 non-null  int64          
 2   patient_fio            85335 non-null  object         
 3   phone                  85335 non-null  object         
 4   email                  36050 non-null  object         
 5   consultation_date      85335 non-null  object         
 6   consultation_duration  85335 non-null  timedelta64[ns]
 7   doctor_fio             85335 non-null  object         
 8   doctor_profession      85335 non-null  object         
 9   rating                 22401 non-null  float64        
 10  comment                8059 non-null   object         
 11  name                   85335 non-null  object         
dtypes: float64(1), int64(2), object(8), timedelta6

In [8]:
from region import get_region_by_two_bases, clean_region

In [9]:
log('getting dirty region by phone number')
df['dirty_region'] = df['phone'].apply(get_region_by_two_bases)

df['region'] = df['dirty_region'].apply(clean_region)
log('cleaning regions')

2020-05-26 15:00:03 getting dirty region by phone number
2020-05-26 15:02:48 cleaning regions


In [10]:
df_tatar = df[(df['region'] == 'Республика Татарстан') & (df['name'] == 'VEBMED')].reset_index(drop=True)

In [11]:
df_tatar['consultation_duration'] = (df_tatar['consultation_duration'] + pd.Timestamp(0))
df_tatar['consultation_duration'] = df_tatar.consultation_duration.apply(lambda x: x.time())

In [12]:
df_tatar.head()

,appoint_id,patient_id,patient_fio,phone,email,consultation_date,consultation_duration,doctor_fio,doctor_profession,rating,comment,name,dirty_region,region
0,230658,3754849,Бадриева Ландыш Раяновна,79083351385,None,2020-05-26,00:00:00.668000,Антипова Оксана Николаевна,Дежурный врач,NaN,None,VEBMED,Республика Татарстан,Республика Татарстан
1,230542,3750668,Иванова Светлана,79093096036,,2020-05-26,00:07:07.598000,Кипкеева Фатима Хусеиновна,Дежурный врач,NaN,None,VEBMED,Республика Татарстан,Республика Татарстан
2,230802,3552937,Хасаншина Зиля Айратовна,79656282777,zilya.airatovna@yandex.ru,2020-05-26,00:20:41.333000,Пономарева Анна Петровна,Дежурный врач,NaN,None,VEBMED,Республика Татарстан,Республика Татарстан
3,230654,3746643,мухомейдшина гузель равильевна,79178739668,None,2020-05-26,00:13:24.479000,Крюченко Оксана Эдуардовна,Эндокринолог,5.0,None,VEBMED,Республика Татарстан,Республика Татарстан
4,228949,3583968,берсенёва рузиля ришатовна,79274176898,ruzilya1627@gmail.com,2020-05-26,00:05:36.180000,Шегай Дмитрий Леонидович,Терапевт,5.0,None,VEBMED,Республика Татарстан,Республика Татарстан


In [13]:
df_tatar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2264 entries, 0 to 2263
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   appoint_id             2264 non-null   int64  
 1   patient_id             2264 non-null   int64  
 2   patient_fio            2264 non-null   object 
 3   phone                  2264 non-null   object 
 4   email                  1446 non-null   object 
 5   consultation_date      2264 non-null   object 
 6   consultation_duration  2264 non-null   object 
 7   doctor_fio             2264 non-null   object 
 8   doctor_profession      2264 non-null   object 
 9   rating                 629 non-null    float64
 10  comment                249 non-null    object 
 11  name                   2264 non-null   object 
 12  dirty_region           2264 non-null   object 
 13  region                 2264 non-null   object 
dtypes: float64(1), int64(2), object(11)
memory usage: 247.8+

In [14]:
# total consultations
df_tatar.shape[0]

2264

In [15]:
# unique patients
len(df_tatar.patient_id.unique())

818

In [16]:
#df_tatar['consultation_duration'] = (df_tatar['consultation_duration'] + pd.Timestamp(0))

In [17]:
#df_tatar['consultation_duration'][77].time()

In [18]:
# df_tatar['consultation_duration'] = df_tatar.consultation_duration.apply(lambda x: x.strftime('%H:%M:%S'))

In [19]:
#df_tatar['consultation_duration'] = df_tatar.consultation_duration.apply(lambda x: x.time())

In [20]:
df_tatar_group = df_tatar.groupby(['patient_fio', 'phone', 'email'])[['patient_id']].count().rename(columns={'patient_id':'consultations'}).reset_index()

In [21]:
df_tatar_group

,patient_fio,phone,email,consultations
0,",Бакеев Искандер Камилевич",79173947803,needik85@gmail.com,1
1,Butorina Irina Aleksandrovna,79377752054,birena1@mail.ru,4
2,Cучкова Ирина Михайловна,79600462199,,8
3,Vakhitova Elvira,79173965987,vakhitovae.1@gmail.com,1
4,Абдуллина Алина Айдаровна,79872336570,alinkaa.1710@gmail.com,5
...,...,...,...,...
470,тимерханова алсу фердинантовна,79393813089,diana-timerhanov@mail.ru,7
471,титова Светлана сергеевна,79393801717,svetlanatitova2028@gmail.com,6
472,хабибуллина айсылу,79274715906,newmoon1247@mail.ru,1
473,хасанова миляуша ринатовна,79050258897,,2


In [22]:
d = dict()

d = {
    'Всего консультаций':df_tatar.shape[0],
    'Уникальных пациентов':len(df_tatar.patient_id.unique())
    
}

In [23]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()
report.columns = ['Количество']

In [24]:
report

,Количество
Всего консультаций,2264
Уникальных пациентов,818


In [25]:
report_date: str = datetime.datetime.now().strftime(DATETIME_FORMAT).replace(':', '-')

In [26]:
global writer
writer = None

In [27]:
def corr_width(x: int):
    if x <= 7:
        return 7
    elif x >=30:
        return 30
    return x

def auto_columns_width(file_name, data, sheet_name, formatting=[]):
    
    # writer
    global writer
    if not writer:
        writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
    
    # drop pandas RangeIndex
    if isinstance(data.index, pd.core.indexes.range.RangeIndex):
        data.to_excel(writer, sheet_name=sheet_name, index=False)
    else:
        data.to_excel(writer, sheet_name=sheet_name)
    
    # select current worksheet
    worksheet = writer.sheets[sheet_name]
    
    # read columns
    cols = list(data.columns)
    cols_iter = list(data.columns)
    
    # datacolumns to str
    for i in range(len(cols)):
        if isinstance(cols[i], datetime.date):
            cols[i] = cols[i].strftime('%Y-%m-%d')
    
    # if index exists = set k=1
    k = 0
    if data.index.dtype == object:
        k = 1
    
    # setting index length
    worksheet.set_column(0, 0, max(data.index.astype(str).map(len)) * 1.5)
    
    # setting columns length
    for i, col in enumerate(cols, k):
        
        if data[cols_iter[i-k]].dtype == int:
            col_len = corr_width(len(col)) * 1.5
        else:
            # эта помойка выбирает что длиннее - название столбца или какое-то значение внутри него
            col_len = max(corr_width(max(data[(cols_iter[i-k])].fillna(0).astype(str).map(len))), len(col))
            col_len *= 1.5
        
        print(i, col, col_len)
        worksheet.set_column(i, i, col_len)

In [28]:
auto_columns_width('Tatarstan_' + report_date + '.xlsx', df_tatar, 'data')
auto_columns_width('Tatarstan_' + report_date + '.xlsx', df_tatar_group, 'Groupby')
auto_columns_width('Tatarstan_' + report_date + '.xlsx', report, 'Cons-Users')

writer.save()

0 appoint_id 15.0
1 patient_id 15.0
2 patient_fio 45.0
3 phone 16.5
4 email 45.0
5 consultation_date 25.5
6 consultation_duration 31.5
7 doctor_fio 45.0
8 doctor_profession 45.0
9 rating 10.5
10 comment 45.0
11 name 10.5
12 dirty_region 30.0
13 region 30.0
0 patient_fio 45.0
1 phone 16.5
2 email 45.0
3 consultations 19.5
1 Количество 15.0


In [29]:
#df_tatar.to_excel('123.xlsx')